In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("API_KEY"), base_url=os.getenv("BASE_URL"))

In [4]:
response = llm.invoke("How will the weather be in munich today?")
response.content

"I'm unable to provide real-time weather updates. For the latest weather information in Munich, I recommend checking a reliable weather website or app."

In [5]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [6]:
llm_with_tools = llm.bind_tools(tools)

In [10]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result.additional_kwargs

{'tool_calls': [{'id': 'chatcmpl-nFkpWQoaT9sIdl2nd4Kv03gvSL5v9',
   'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'},
   'type': 'function'}],
 'refusal': None}

In [11]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'chatcmpl-nFkpWQoaT9sIdl2nd4Kv03gvSL5v9',
  'type': 'tool_call'}]

In [12]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-r4GrijVEVCJoUr5Bp1i7GUceHvmkC', 'function': {'arguments': '{"location": "Munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'chatcmpl-r4GrijVEVCJoUr5Bp1i7GUceHvmkC', 'function': {'arguments': '{"location": "Munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 90, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_57db37749c', 'id': 'chatcmpl-r4GrijVEVCJoUr5Bp1i7GUceHvmkC', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--75f3c957-cbfc-4e45-8759-c5f241b6a22d-0', tool_calls=[{'name': 'get_weather', 'args': {

In [13]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'chatcmpl-r4GrijVEVCJoUr5Bp1i7GUceHvmkC',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Munich', 'seating_type': 'outdoor'},
  'id': 'chatcmpl-r4GrijVEVCJoUr5Bp1i7GUceHvmkC',
  'type': 'tool_call'}]

In [14]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [15]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE', 'function': {'arguments': '{"location": "Munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE', 'function': {'arguments': '{"location": "Munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 90, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_57db37749c', 'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE', 'service_tie

In [16]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [17]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Munich', 'seating_type': 'outdoor'},
  'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE',
  'type': 'tool_call'}]

In [18]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [19]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE', 'function': {'arguments': '{"location": "Munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE', 'function': {'arguments': '{"location": "Munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 90, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_57db37749c', 'id': 'chatcmpl-gGMvi4Wfas2bi2MtDoT7V6Du2rqxE', 'service_tie

In [20]:
llm_with_tools.invoke(messages)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'. (request id: 20250708172927507231049VKK0pvit) (request id: 20250708172924948000226NCeK9Tmw)", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}